In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [4]:
# Filter DataFrame for total_votes greater or equal to 20
total_votes_df = df.filter("total_votes >= 20")
total_votes_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|   26252807|R2P76PJFUGXBHO|B013S8HI5U|      22756004|Lovenest Men's- S...|         Apparel|          1|    

In [5]:
# Filter DataFrame for helpful_votes/total_votes >= 50%
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes >= 0.5")
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|     453792| RQW4AFOG9MR4Z|B013QTWRAC|     254612789|UONBOX Celebrity ...|         Apparel|          5|    

In [6]:
# Filter DataFrame for paid program reviews
paid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51744990| R6U9701C3BGO6|B00XL2X3U8|     139556466|Wrangler Authenti...|         Apparel|          3|          139|        147|   Y|                N|        Comfort-Lite|After some debate...| 2015-08-31|
|         US|   35360512|R1XK3ALB45D7N4|B00XKYRLEQ|     243221870|Wrangler Authenti...|         Apparel|          5|    

In [7]:
# Filter DataFrame for unpaid program reviews
unpaid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|     453792| RQW4AFOG9MR4Z|B013QTWRAC|     254612789|UONBOX Celebrity ...|         Apparel|          5|    

In [8]:
from pyspark.sql.functions import count

In [9]:
# Total number of paid reviews
paid_reviews = paid_review_df.count()
paid_reviews

33

In [10]:
# Total number of unpaid reviews
unpaid_reviews = unpaid_review_df.count()
unpaid_reviews

45388

In [15]:
# Total number of paid 5 star reviews
five_star_paid = paid_review_df.filter(paid_review_df["star_rating"] == "5").count()
five_star_paid

15

In [16]:
# Total number of unpaid 5 star reviews
five_star_unpaid = unpaid_review_df.filter(unpaid_review_df["star_rating"] == "5").count()
five_star_unpaid

23733

In [17]:
# Percentage of paid 5 star reviews
five_star_paid_percent = (five_star_paid/paid_reviews) * 100
five_star_paid_percent

45.45454545454545

In [18]:
# Percentage of unpaid 5 star reviews
fiveStar_unpaid_percent = (five_star_unpaid/unpaid_reviews) * 100
fiveStar_unpaid_percent

52.28915131752886